In [124]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

OriginURL = r"https://sh.lianjia.com/chengjiao/"
RequestHeader ={
"Host": "sh.lianjia.com",
"Connection": "keep-alive",
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36",
"Accept": "image/webp,image/apng,image/*,*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "zh-CN,zh;q=0.9"}

ContentOrigin = requests.get(OriginURL)
BSOrigin = BeautifulSoup(ContentOrigin.text, "lxml")
DistrictListOrigin = BSOrigin.find_all("a",href=re.compile("/chengjiao/\w+/"),title=re.compile("上海"))
DistrictList = ["https://sh.lianjia.com"+ i["href"] for i in DistrictListOrigin]


In [123]:
def getBlockList(URL):
    _connection = requests.get(URL)
    _BS = BeautifulSoup(_connection.text, "lxml")
    
    return _BlockList
getBlockList("https://sh.lianjia.com/chengjiao/pudong/")

['https://sh.lianjia.com/chengjiao/pudong/',
 'https://sh.lianjia.com/chengjiao/minhang/',
 'https://sh.lianjia.com/chengjiao/baoshan/',
 'https://sh.lianjia.com/chengjiao/xuhui/',
 'https://sh.lianjia.com/chengjiao/putuo/',
 'https://sh.lianjia.com/chengjiao/yangpu/',
 'https://sh.lianjia.com/chengjiao/changning/',
 'https://sh.lianjia.com/chengjiao/songjiang/',
 'https://sh.lianjia.com/chengjiao/jiading/',
 'https://sh.lianjia.com/chengjiao/huangpu/',
 'https://sh.lianjia.com/chengjiao/jingan/',
 'https://sh.lianjia.com/chengjiao/zhabei/',
 'https://sh.lianjia.com/chengjiao/hongkou/',
 'https://sh.lianjia.com/chengjiao/qingpu/',
 'https://sh.lianjia.com/chengjiao/fengxian/',
 'https://sh.lianjia.com/chengjiao/jinshan/',
 'https://sh.lianjia.com/chengjiao/chongming/',
 'https://sh.lianjia.com/chengjiao/shanghaizhoubian/']

In [125]:
URL = "https://sh.lianjia.com/chengjiao/pudong/"
_connection = requests.get(URL)
_BS = BeautifulSoup(_connection.text, "lxml")




In [135]:
_BS.find_all("div")

[<div class="banner"><div class="container"><ul class="channelList"><li><a href="//www.lianjia.com/">首页</a></li><li class="selected"><a href="https://sh.lianjia.com/ershoufang/">二手房</a></li><li class=""><a href="https://sh.fang.lianjia.com/">新房</a></li><li class=""><a href="https://sh.lianjia.com/zufang/">租房</a></li><li class="" rel="nofollow"><a href="http://you.lianjia.com/">旅居</a></li><li class=""><a href="https://us.lianjia.com">海外</a></li><li class=""><a href="https://sh.lianjia.com/xiaoqu/">小区</a></li><li class=""><a href="https://sh.lianjia.com/jingjiren/">经纪人</a></li><li class=""><a href="https://sh.lianjia.com/wenda/">指南</a><div class="childList"><a href="https://sh.lianjia.com/wenda/">问答</a><a href="https://news.lianjia.com/sh/baike/">百科</a></div></li><li class=""><a href="https://sh.lianjia.com/tool.html" target="_blank">工具</a></li><li class=""><a href="https://sh.lianjia.com/yezhu/" target="_blank">发布房源</a></li></ul><div class="banner-right"><div class="login" id="userInfoC